# Hardware Test: Logical State-Dependent Error (LRT Prediction)

## Real IBM Quantum Hardware Validation

**Copyright © 2025 James D. (JD) Longmire**  
**License**: Apache License 2.0  
**Citation**: Longmire, J.D. (2025). *Logic Realism Theory: A Research Program for Ontological Logic in Informational Reality*. Logic Realism Theory Repository.

---

## Purpose

This notebook runs the **first real hardware test** of Logic Realism Theory predictions.

**Test**: Logical State-Dependent Error  
**Prediction**: Superposition states |+⟩ may have excess error beyond standard T2 decoherence  
**Method**: Ramsey experiment on IBM ibm_torino (133-qubit processor)  
**Budget**: 25 points × 2,500 shots = 62,500 shots (~5-7 minutes of quantum time)

---

## Test Design

**Parent Document**: `theory/predictions/Logical_State_Dependent_Error_Test_Design.md`  
**Phase 3 Validation**: `theory/predictions/Phase_3_Results_Report.md` (VIF = 1.0, MDE = 0.5%)

**Status**: Hardware-ready (multi-LLM approved, quality 0.69, Phase 3 complete)

---

## ⚠️ IMPORTANT: This Will Use Real Quantum Time

- Estimated runtime: **5-7 minutes** of ibm_torino processor time
- Queue wait: **Variable** (413 pending jobs as of 2025-10-26)
- Budget remaining after: ~3-5 minutes for retries/analysis

**Confirmation required before submission (see Step 5)**

---

## Step 1: Environment Setup

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import curve_fit
import time
import json
from datetime import datetime

# Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler

print("Environment setup complete")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Step 2: Connect to IBM Quantum

In [ ]:
# Load API token
with open('../../theory/predictions/IBM_Q_API.txt', 'r') as f:
    token = f.read().strip()

# Connect to IBM Quantum
print("Connecting to IBM Quantum Platform...")
try:
    service = QiskitRuntimeService(channel="ibm_quantum_platform", token=token)
    print("Connected successfully (token loaded)")
except:
    # If already saved
    service = QiskitRuntimeService(channel="ibm_quantum_platform")
    print("Connected successfully (saved credentials)")

# Get backend
backend = service.backend('ibm_torino')
print(f"\nBackend: {backend.name}")
print(f"Status: {backend.status().status_msg}")
print(f"Queue: {backend.status().pending_jobs} pending jobs")
print(f"Qubits: {backend.configuration().n_qubits}")
print(f"Max shots: {backend.configuration().max_shots:,}")

## Step 3: Test Parameters

In [ ]:
# Hardware test parameters (reduced for 10-minute budget)
N_SHOTS = 2500  # Shots per circuit (vs Phase 3: 10,000)
N_POINTS = 25   # Duration points (vs Phase 3: 49)

# Duration sweep (will be calibrated to measured T2)
# Start with typical IBM range: T2 ~ 60-150 us
T2_ESTIMATE = 100e-6  # 100 us (conservative estimate)

# Log + Linear sampling (optimized from Phase 3)
T_MIN = 1e-6  # 1 us
T_MAX = 5 * T2_ESTIMATE  # 500 us (5*T2)

# Generate duration points
T_log = np.logspace(np.log10(T_MIN), np.log10(T2_ESTIMATE), 13)  # 1 us to T2
T_lin = np.linspace(T2_ESTIMATE, T_MAX, 13)  # T2 to 5*T2
T_sweep = np.sort(np.concatenate([T_log, T_lin[1:]]))  # 25 points total

print("Hardware Test Parameters:")
print(f"  Shots per circuit: {N_SHOTS:,}")
print(f"  Duration points: {N_POINTS}")
print(f"  Total shots: {N_SHOTS * N_POINTS:,}")
print(f"  Duration range: {T_sweep.min()*1e6:.2f} to {T_sweep.max()*1e6:.1f} us")
print(f"  Estimated T2: {T2_ESTIMATE*1e6:.0f} us")
print(f"\nEstimated quantum time: ~5-7 minutes")
print(f"Budget remaining after: ~3-5 minutes")

## Step 4: Build Ramsey Circuits

In [ ]:
def build_ramsey_circuit(duration_sec, qubit=0):
    """
    Build Ramsey experiment circuit.
    
    Circuit: H - delay(T) - H - Measure
    Prepares |+⟩, waits duration T, measures in X-basis.
    
    Args:
        duration_sec: Wait time in seconds
        qubit: Which qubit to use (default 0)
    
    Returns:
        QuantumCircuit
    """
    qc = QuantumCircuit(1, 1)
    
    # Prepare |+⟩ (superposition state)
    qc.h(0)
    
    # Wait (decoherence happens here)
    qc.delay(duration_sec, 0, unit='s')
    
    # Measure in X-basis (rotate back with H)
    qc.h(0)
    qc.measure(0, 0)
    
    return qc

# Build all circuits
print("Building Ramsey circuits...")
circuits = []
for T in T_sweep:
    qc = build_ramsey_circuit(T, qubit=0)
    circuits.append(qc)

print(f"Built {len(circuits)} circuits")

# Show example circuit (shortest duration)
print("\nExample circuit (T = 1 us):")
print(circuits[0])

## Step 5: Transpile for Hardware

In [ ]:
# Transpile circuits for ibm_torino
print("Transpiling circuits for ibm_torino...")
print("This may take 1-2 minutes...")

start_time = time.time()
transpiled_circuits = transpile(
    circuits,
    backend=backend,
    optimization_level=3,  # Maximum optimization
    initial_layout=[0],     # Use qubit 0
)
elapsed = time.time() - start_time

print(f"Transpilation complete ({elapsed:.1f}s)")
print(f"Total transpiled circuits: {len(transpiled_circuits)}")

# Show gate counts
example = transpiled_circuits[0]
print(f"\nExample transpiled circuit (T = 1 us):")
print(f"  Depth: {example.depth()}")
print(f"  Gate count: {example.count_ops()}")

## Step 6: ⚠️ SUBMIT TO HARDWARE (Confirmation Required)

**This will use ~5-7 minutes of quantum time.**

**Before running this cell, confirm:**
- ✅ You understand this uses real quantum time
- ✅ Queue is acceptable (check status above)
- ✅ You're ready to wait for results

**To proceed: Change `CONFIRMED = False` to `CONFIRMED = True` below, then run.**

In [ ]:
# Safety: Require explicit confirmation
CONFIRMED = False  # Change to True to proceed

if not CONFIRMED:
    print("⚠️ SUBMISSION NOT CONFIRMED")
    print("To submit to hardware, change CONFIRMED = True above and re-run this cell.")
    print("\nThis will:")
    print(f"  - Submit {len(transpiled_circuits)} circuits to ibm_torino")
    print(f"  - Run {N_SHOTS:,} shots per circuit")
    print(f"  - Use ~5-7 minutes of quantum time")
    print(f"  - Current queue: {backend.status().pending_jobs} jobs")
else:
    print("✅ CONFIRMED - Submitting to hardware...")
    print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Queue status: {backend.status().pending_jobs} pending jobs")
    print("\nOpening session...")
    
    # Open session and submit
    with Session(backend=backend) as session:
        sampler = Sampler(session=session)
        
        print("Submitting job...")
        job = sampler.run(
            transpiled_circuits,
            shots=N_SHOTS
        )
        
        job_id = job.job_id()
        print(f"\n✅ Job submitted successfully!")
        print(f"Job ID: {job_id}")
        print(f"Status: {job.status()}")
        
        # Save job ID for recovery
        with open('hardware_test_job_id.txt', 'w') as f:
            f.write(job_id)
        print(f"\nJob ID saved to: hardware_test_job_id.txt")
        
        # Monitor job
        print("\nWaiting for results (this may take time depending on queue)...")
        print("You can close this notebook and retrieve results later using the Job ID.")
        print("\nMonitoring job status...")
        
        while job.status() not in ['DONE', 'CANCELLED', 'ERROR']:
            status = job.status()
            print(f"  Status: {status} - {datetime.now().strftime('%H:%M:%S')}")
            time.sleep(30)  # Check every 30 seconds
        
        final_status = job.status()
        print(f"\n✅ Job completed: {final_status}")
        print(f"Completion time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        if final_status == 'DONE':
            # Retrieve results
            print("\nRetrieving results...")
            result = job.result()
            
            # Save raw results
            print("Saving raw results...")
            with open('hardware_test_results.json', 'w') as f:
                json.dump({
                    'job_id': job_id,
                    'backend': backend.name,
                    'n_shots': N_SHOTS,
                    'n_points': N_POINTS,
                    'T_sweep': T_sweep.tolist(),
                    'timestamp': datetime.now().isoformat(),
                    'status': str(final_status)
                }, f, indent=2)
            
            print("\n✅ Results saved successfully!")
            print("Files created:")
            print("  - hardware_test_job_id.txt")
            print("  - hardware_test_results.json")
            print("\nProceed to Step 7 to analyze results.")
        else:
            print(f"\n❌ Job failed with status: {final_status}")
            print("Check IBM Quantum dashboard for details.")

## Step 7: Process Hardware Results

In [ ]:
# Load results (either from previous cell or from saved file)
print("Processing hardware results...")

if 'result' not in locals():
    print("Loading results from file...")
    # Reconnect and retrieve job
    with open('hardware_test_job_id.txt', 'r') as f:
        job_id = f.read().strip()
    
    job = service.job(job_id)
    result = job.result()
    print(f"Results loaded for job: {job_id}")

# Extract measurement counts
results_data = []
for i, T in enumerate(T_sweep):
    # Get counts for this circuit
    counts = result[i].data.c.get_counts()
    
    # Calculate probabilities
    counts_0 = counts.get(0, 0)  # |0⟩ (logical |+⟩)
    counts_1 = counts.get(1, 0)  # |1⟩ (logical |-⟩, error)
    total = counts_0 + counts_1
    
    p_error = counts_1 / total if total > 0 else 0
    
    results_data.append({
        'T': T,
        'T_us': T * 1e6,
        'counts_0': counts_0,
        'counts_1': counts_1,
        'p_error': p_error,
        'shots': total
    })

df_hardware = pd.DataFrame(results_data)
print(f"\nProcessed {len(df_hardware)} data points")
print(f"Total shots collected: {df_hardware['shots'].sum():,}")
print("\nFirst 5 data points:")
print(df_hardware[['T_us', 'p_error', 'shots']].head())

## Step 8: T2 Characterization from Hardware Data

In [ ]:
def ramsey_model(T, A, T2):
    """Ramsey decay: p_error(T) = A * (1 - exp(-T/T2))"""
    return A * (1 - np.exp(-T / T2))

# Fit T2 from hardware data
print("Fitting T2 from hardware measurements...")

try:
    popt, pcov = curve_fit(
        ramsey_model,
        df_hardware['T'],
        df_hardware['p_error'],
        p0=[0.5, T2_ESTIMATE],
        bounds=([0, 1e-6], [1, 1e-3]),  # Reasonable bounds
        maxfev=10000
    )
    
    A_fit, T2_fit = popt
    
    print(f"\n✅ T2 Characterization Results:")
    print(f"  Fitted T2: {T2_fit*1e6:.2f} us")
    print(f"  Fitted amplitude: {A_fit:.4f}")
    print(f"  Expected amplitude: ~0.5")
    
    # Generate QM baseline prediction
    df_hardware['p_predicted'] = ramsey_model(df_hardware['T'], A_fit, T2_fit)
    df_hardware['residual'] = df_hardware['p_error'] - df_hardware['p_predicted']
    
    # Calculate R²
    ss_res = np.sum(df_hardware['residual']**2)
    ss_tot = np.sum((df_hardware['p_error'] - df_hardware['p_error'].mean())**2)
    R2 = 1 - (ss_res / ss_tot)
    
    print(f"  R²: {R2:.4f} (QM fit quality)")
    
except Exception as e:
    print(f"\n❌ Fit failed: {e}")
    print("Using estimated T2 for baseline.")
    T2_fit = T2_ESTIMATE
    A_fit = 0.5
    df_hardware['p_predicted'] = ramsey_model(df_hardware['T'], A_fit, T2_fit)
    df_hardware['residual'] = df_hardware['p_error'] - df_hardware['p_predicted']

## Step 9: Residual Analysis - LRT Test

In [ ]:
print("=" * 70)
print("LRT TEST: Residual Analysis")
print("=" * 70)

# Regression: Δp(T) = β_LRT * T + ε
slope, intercept, r_value, p_value, std_err = linregress(
    df_hardware['T'],
    df_hardware['residual']
)

print(f"\nRegression Results:")
print(f"  β_LRT (slope): {slope:.6e} ± {std_err:.6e}")
print(f"  Intercept:     {intercept:.6e}")
print(f"  R²:            {r_value**2:.4f}")
print(f"  p-value:       {p_value:.4f}")

print(f"\n" + "=" * 70)
if p_value < 0.05:
    excess_pct = slope * T2_fit * 100  # Excess error at T = T2
    print("🔴 LRT SIGNAL DETECTED")
    print("=" * 70)
    print(f"\nStatistically significant excess error detected!")
    print(f"  Excess error at T=T2: {excess_pct:.2f}%")
    print(f"  Confidence: p = {p_value:.4f} < 0.05")
    print(f"\nThis supports LRT prediction that superposition states")
    print(f"have excess error beyond standard QM decoherence.")
else:
    print("🟢 NO LRT SIGNAL DETECTED")
    print("=" * 70)
    print(f"\nNo statistically significant excess error.")
    print(f"  p-value: {p_value:.4f} > 0.05")
    print(f"  β_LRT: {slope:.6e} (consistent with 0)")
    print(f"\nResult is consistent with standard QM.")
    print(f"LRT prediction not supported by this data.")

print("\n" + "=" * 70)

# Statistical summary
print(f"\nResidual Statistics:")
print(f"  Mean:   {df_hardware['residual'].mean():.6e}")
print(f"  Std:    {df_hardware['residual'].std():.6e}")
print(f"  Min:    {df_hardware['residual'].min():.6e}")
print(f"  Max:    {df_hardware['residual'].max():.6e}")

## Step 10: Visualization

In [ ]:
# Create comprehensive visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Panel A: T2 Characterization
ax1.scatter(df_hardware['T_us'], df_hardware['p_error'], 
            alpha=0.6, s=50, label=f'Hardware Data (N={N_SHOTS} shots)')

T_fine = np.linspace(df_hardware['T'].min(), df_hardware['T'].max(), 200)
p_fine = ramsey_model(T_fine, A_fit, T2_fit)
ax1.plot(T_fine * 1e6, p_fine, 'r-', linewidth=2, 
         label=f'QM Fit (T2={T2_fit*1e6:.1f} us, R²={R2:.4f})')

ax1.set_xlabel('Duration T (us)', fontsize=12)
ax1.set_ylabel('Error Probability p(|-⟩)', fontsize=12)
ax1.set_title('Panel A: T2 Characterization via Ramsey Experiment', 
              fontsize=14, fontweight='bold')
ax1.legend(loc='lower right', fontsize=10)
ax1.grid(True, alpha=0.3)

# Panel B: Residual Analysis (LRT Test)
ax2.scatter(df_hardware['T_us'], df_hardware['residual'], 
            alpha=0.6, s=50, color='blue', label='Residuals')

# Regression line
T_fit = np.array([df_hardware['T'].min(), df_hardware['T'].max()])
residual_fit = slope * T_fit + intercept
ax2.plot(T_fit * 1e6, residual_fit, 'r--', linewidth=2, 
         label=f'Linear Fit: β={slope:.2e}, p={p_value:.4f}')

ax2.axhline(0, color='k', linestyle=':', alpha=0.5)

ax2.set_xlabel('Duration T (us)', fontsize=12)
ax2.set_ylabel('Residual Δp = p_observed - p_QM', fontsize=12)

if p_value < 0.05:
    ax2.set_title('Panel B: LRT Test - SIGNAL DETECTED', 
                  fontsize=14, fontweight='bold', color='red')
else:
    ax2.set_title('Panel B: LRT Test - No Signal (QM Consistent)', 
                  fontsize=14, fontweight='bold', color='green')

ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('hardware_lrt_test_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Figure saved: hardware_lrt_test_results.png")

## Step 11: Save Final Results

In [ ]:
# Save comprehensive results report
report = {
    'metadata': {
        'backend': backend.name,
        'job_id': job_id if 'job_id' in locals() else 'N/A',
        'timestamp': datetime.now().isoformat(),
        'n_shots': N_SHOTS,
        'n_points': N_POINTS,
        'total_shots': N_SHOTS * N_POINTS
    },
    'T2_characterization': {
        'T2_fitted_us': float(T2_fit * 1e6),
        'amplitude': float(A_fit),
        'R2': float(R2) if 'R2' in locals() else None
    },
    'LRT_test': {
        'beta_LRT': float(slope),
        'beta_LRT_stderr': float(std_err),
        'intercept': float(intercept),
        'R2': float(r_value**2),
        'p_value': float(p_value),
        'signal_detected': p_value < 0.05,
        'excess_error_pct_at_T2': float(slope * T2_fit * 100) if p_value < 0.05 else 0
    },
    'residual_stats': {
        'mean': float(df_hardware['residual'].mean()),
        'std': float(df_hardware['residual'].std()),
        'min': float(df_hardware['residual'].min()),
        'max': float(df_hardware['residual'].max())
    },
    'data': df_hardware.to_dict('records')
}

with open('hardware_lrt_test_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("✅ Results saved to: hardware_lrt_test_report.json")
print("\nFinal Summary:")
print(f"  Backend: {backend.name}")
print(f"  Total shots: {N_SHOTS * N_POINTS:,}")
print(f"  T2 (fitted): {T2_fit*1e6:.2f} us")
print(f"  β_LRT: {slope:.2e} ± {std_err:.2e}")
print(f"  p-value: {p_value:.4f}")
print(f"  LRT signal: {'YES' if p_value < 0.05 else 'NO'}")

---

## Summary

This notebook implements the first real hardware test of Logic Realism Theory predictions.

**Test Design Validated**: Multi-LLM approved (quality 0.69), Phase 3 simulation validated (VIF = 1.0, MDE = 0.5%)

**Hardware Test**: IBM ibm_torino (133-qubit processor)

**Analysis**: Residual analysis framework isolates LRT effect from standard QM baseline

**Next Steps**:
- Review results in `hardware_lrt_test_report.json`
- Check visualization in `hardware_lrt_test_results.png`
- Document findings in session log
- If LRT signal detected: Write up results for paper
- If no signal: Document upper bounds, consider alternative tests

---

**Files Created**:
- `hardware_test_job_id.txt` - Job ID for recovery
- `hardware_test_results.json` - Raw measurement data
- `hardware_lrt_test_report.json` - Comprehensive analysis
- `hardware_lrt_test_results.png` - Visualization

---